In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get -y install protobuf-compiler python-pil python-lxml

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python-pil
E: Unable to locate package python-lxml


In [ ]:
!pip install openNMT-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 25.6 MB/s eta 0:00:00


In [ ]:
config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src: /content/drive/MyDrive/EngSwaBBC/cleanGoURMETEng.txt.subword.train
        path_tgt: /content/drive/MyDrive/EngSwaBBC/cleanGoURMETSwa.txt.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: /content/drive/MyDrive/EngSwaBBC/cleanGoURMETEng.txt.subword.dev
        path_tgt: /content/drive/MyDrive/EngSwaBBC/cleanGoURMETSwa.txt.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 64
tgt_seq_length: 64

# Tokenization options
src_subword_model: /content/drive/MyDrive/EngSwaBBC/source.model
tgt_subword_model: /content/drive/MyDrive/EngSwaBBC/target.model
# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.engswa

# Stop training if it does not imporve after n validations
early_stopping: 10

# Default: 5000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 2023

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 15000

# Default: 10000 - Run validation after n steps
valid_steps: 4000

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 5000
report_every: 100

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 1024  # Tokens per batch, change when CUDA out of memory
valid_batch_size: 1024
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.4
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 16
dec_layers: 16
heads: 16
hidden_size: 1024
word_vec_size: 1024
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!onmt_build_vocab -config /content/config.yaml -n_sample -1 -num_threads 2

2023-09-22 01:41:09.135623: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 01:41:12.360778: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-22 01:41:14.603021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 01:41:14.603471: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
 # Train the NMT model
!onmt_train -config /content/config.yaml

2023-09-22 01:41:29.446119: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 01:41:30.489484: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-22 01:41:31.952484: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 01:41:31.952899: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
!onmt_translate -model /content/models/model.engswa_step_15000.pt -src /content/drive/MyDrive/EngSwaBBC/cleanGoURMETEng.txt.subword.test -output /content/drive/MyDrive/EngSwaBBC/GoURMETSwa_Translated_V4

2023-09-22 05:10:49.042187: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 05:10:51.058130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-22 05:10:54.019271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-22 05:10:54.019793: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
!mkdir -p nmt
%cd nmt
!git clone https://github.com/ymoslem/MT-Preparation.git

/content/nmt
Cloning into 'MT-Preparation'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (248/248), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 248 (delta 123), reused 193 (delta 97), pack-reused 0
Receiving objects: 100% (248/248), 64.32 KiB | 4.02 MiB/s, done.
Resolving deltas: 100% (123/123), done.


In [ ]:
# Install the requirements
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
%cd  ../

In [ ]:
!ls

In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/EngSwaBBC/target.model /content/drive/MyDrive/EngSwaBBC/GoURMETSwa_Translated_V4

In [ ]:
!python3 nmt/MT-Preparation/subwording/3-desubword.py /content/drive/MyDrive/EngSwaBBC/target.model /content/drive/MyDrive/EngSwaBBC/cleanGoURMETSwa.txt.subword.test

In [ ]:

# Download the BLEU script
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
!pip3 install sacrebleu

--2023-09-22 07:00:09--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py.1’

compute-bleu.py.1   100%[===================>]     957  --.-KB/s    in 0s      

2023-09-22 07:00:09 (45.6 MB/s) - ‘compute-bleu.py.1’ saved [957/957]



In [ ]:
!python3 compute-bleu.py /content/drive/MyDrive/EngSwaBBC/cleanGoURMETSwa.txt.subword.test.desubword /content/drive/MyDrive/EngSwaBBC/GoURMETSwa_Translated_V4.desubword

Reference 1st sentence: Wale walioingia kwenye safina waliokolewa, wakati wale waliokataa kufuata maelekezo ya Mungu waliachwa nje.
MTed 1st sentence: Wale walioingia kwenye safina waliokolewa, wakati wale waliokataa kufuata maelekezo ya Mungu waliachwa nje.
BLEU:  37.08511655662515
